
Demos - Geo - Plotly and streamlit
==================================


* [Medium - How to Create a Simple GIS Map with Plotly and Streamlit](https://towardsdatascience.com/how-to-create-a-simple-gis-map-with-plotly-and-streamlit-7732d67b84e2)
  + Date: Dec. 2023
  + Author: Alan Jones
  ([Alan Jones on Medium](https://medium.com/@alan-jones),
  [Alan Jones on LinkedIn](https://www.linkedin.com/in/alan-jones-032699100/))
  + Publisher: Medium TowardsDataScience
  + [GitHub - repository with the code for that notebook](https://github.com/alanjones2/st-choropleth/tree/main)
* [GitHub - Data Engineering helpers - Material about geo-analysis with Python](https://github.com/data-engineering-helpers/ks-cheat-sheets/blob/main/programming/python/geo/README.md)
* [Demos - Geo - Plotly and streamlit (this notebook)](https://github.com/data-engineering-helpers/databricks-examples/blob/main/ipython-notebooks/demos-geo-plotly-and-streamlit.ipynb)

In [0]:
%pip install streamlit plotly

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 80.9 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Not uninstalling pygments at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-d16b1a70-2dbc-4953-a70b-29

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd # pyspark.pandas as pd
import streamlit as st
import plotly.express as px
import json


# Data
Our final dashboard will use data about CO2 emissions. This data is derived from tables
in a GitHub repository belonging to
[Our World in Data](https://ourworldindata.org/).
OWID is a great source of data and analysis which I have used many times
(_e.g._ [New Data Demonstrates that 2023 was the Hottest Summer Ever](https://medium.com/towards-data-science/new-data-demonstrates-that-2023-was-the-hottest-summer-ever-d92d500a8f01)).

The data that I have copied from OWID represents the CO2 emissions for countries since 1750.
The original data contains far more information than we need, so I have created subsets
of the data and stored them in the app. You will find them in the data folder.


In [0]:
%sh

rm -rf /dbfs/tmp/demo-streamlit
mkdir -p /dbfs/tmp/demo-streamlit/{data,geo}
curl -kL https://github.com/alanjones2/st-choropleth/raw/main/data/co2_total.csv -o /dbfs/tmp/demo-streamlit/data/co2_total.csv
curl -kL https://github.com/alanjones2/st-choropleth/raw/main/data/Australian%20Bureau%20of%20Statistics.csv -o /dbfs/tmp/demo-streamlit/data/Australian-Bureau-of-Statistics.csv
head -5 /dbfs/tmp/demo-streamlit/data/co2_total.csv
curl -kL https://github.com/alanjones2/st-choropleth/raw/main/geo/australia.geojson -o /dbfs/tmp/demo-streamlit/geo/australia.geojson
ls -lFhR /dbfs/tmp/demo-streamlit


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  806k  100  806k    0     0  1657k      0 --:--:-- --:--:-- --:--:-- 25.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   468  100   468    0     0    895      0 --:--:-- --:--:-- --:--:--   895


,Entity,Code,Year,Annual CO₂ emissions
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  622k  100  622k    0     0  1567k      0 --:--:-- --:--:-- --:--:-- 1567k


/dbfs/tmp/demo-streamlit:
total 8.0K
drwxrwxrwx 2 nobody nogroup 4.0K Dec 28 18:04 data/
drwxrwxrwx 2 nobody nogroup 4.0K Dec 28 18:04 geo/

/dbfs/tmp/demo-streamlit/data:
total 808K
-rwxrwxrwx 1 nobody nogroup  468 Dec 28  2023 Australian-Bureau-of-Statistics.csv*
-rwxrwxrwx 1 nobody nogroup 807K Dec 28 18:04 co2_total.csv*

/dbfs/tmp/demo-streamlit/geo:
total 623K
-rwxrwxrwx 1 nobody nogroup 623K Dec 28  2023 australia.geojson*


In [0]:
df_total = pd.read_csv("/dbfs/tmp/demo-streamlit/data/co2_total.csv")
df_total_2021 = df_total[df_total['Year']==2021]

In [0]:
col = 'Annual CO₂ emissions'
max = df_total_2021[col].max()
min = df_total_2021[col].min()

In [0]:
fig = px.choropleth(df_total_2021, 
                    locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max)
                    )

In [0]:
fig = px.choropleth(df_total_2021, 
                    locations="Code",
                    scope = 'europe',
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    title = 'Europe'
                    )

In [0]:
st.title("Population of Australian States")
st.info("Hover over the map to see the names of the states and their population")
f = open('/dbfs/tmp/demo-streamlit/geo/australia.geojson')
oz = json.load(f)

df = pd.read_csv('/dbfs/tmp/demo-streamlit/data/Australian-Bureau-of-Statistics.csv')
fig = px.choropleth(df, geojson=oz, 
                    color="Population at 31 March 2023 ('000)",
                    locations="State", 
                    featureidkey="properties.name",
                    color_continuous_scale="Reds",
                    range_color=(0, 10000),
                    fitbounds = 'geojson',
                    template = 'plotly_white'
                   )
st.plotly_chart(fig)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-4483247640063516>, line 7
      4 oz = json.load(f)
      6 df = pd.read_csv('/dbfs/tmp/demo-streamlit/data/Australian-Bureau-of-Statistics.csv')
----> 7 fig = px.choropleth(df, geojson=oz, 
      8                     color="Population at 31 March 2023 ('000)",
      9                     locations="State", 
     10                     featureidkey="properties.name",
     11                     color_continuous_scale="Reds",
     12                     range_color=(0, 10000),
     13                     fitbounds = 'geojson',
     14                     template = 'plotly_white'
     15                    )
     16 st.plotly_chart(fig)

File /databricks/python/lib/python3.10/site-packages/plotly/express/_chart_types.py:1091, in choropleth(data_frame, lat, lon, locations, locationmode, geojson, featureidkey, color, facet_r

In [0]:
df = pd.read_csv('data/europop.csv')

fig = px.scatter_geo(df, scope = 'europe', 
                    color="Population (historical estimates)",
                    size="Population (historical estimates)",
                    locations="Code", 
                    hover_name = 'Entity',
                    color_continuous_scale="Purples",
                    range_color=(0, 100000000),
                    fitbounds = 'locations',
                    template = 'plotly_white',
                    title = "European populations"
                   )

st.plotly_chart(fig)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-4483247640063517>, line 1
----> 1 df = pd.read_csv('data/europop.csv')
      3 fig = px.scatter_geo(df, scope = 'europe', 
      4                     color="Population (historical estimates)",
      5                     size="Population (historical estimates)",
   (...)
     12                     title = "European populations"
     13                    )
     15 st.plotly_chart(fig)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/pandas/namespace.py:355, in read

In [0]:
fig = px.choropleth(df_total[df_total['Year']==year], 
                    locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    scope= 'world',
                    projection="orthographic",
                    color_continuous_scale=px.colors.sequential.Reds)
st.plotly_chart(fig)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4483247640063522>, line 1
----> 1 fig = px.choropleth(df_total[df_total['Year']==year], 
      2                     locations="Code",
      3                     color=col,
      4                     hover_name="Entity",
      5                     range_color=(min,max),
      6                     scope= 'world',
      7                     projection="orthographic",
      8                     color_continuous_scale=px.colors.sequential.Reds)
      9 st.plotly_chart(fig)

NameError: name 'df_total' is not defined

In [0]:
st.set_page_config(layout="wide")

st.title("CO2 Emissions")
st.write("""The following maps display the CO2 emissions for a
            range of countries over a range of time""")

st.info("""Use the slider to select a year to display
           the total emissions for each country. 
           Scroll down to see an interactive 3D representation.""")

col1, col2 = st.columns(2)

df_total = pd.read_csv('data/co2_total.csv')
col = 'Annual CO₂ emissions'
max = df_total[col].max()
min = df_total[col].min()

# To get the whole range replace 1950 with the comment that follows it
first_year = 1950 #df_total['Year'].min()
last_year = df_total['Year'].max()
year = st.slider('Select year',first_year,last_year, key=col)

col1.write("""This map uses the 'Natural Earth' projection""")
fig = px.choropleth(df_total[df_total['Year']==year], 
                    locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    scope= 'world',
                    projection="natural earth",
                    color_continuous_scale=px.colors.sequential.Reds)
col1.plotly_chart(fig)

col2.write("""This map uses the 'Orthographic' projection.
Click on the globe and move the pointer to rotate it.
""")

fig = px.choropleth(df_total[df_total['Year']==year], locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    scope= 'world',
                    projection="orthographic",
                    color_continuous_scale=px.colors.sequential.Reds)
col2.plotly_chart(fig)

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-4483247640063523>, line 13
      7 st.info("""Use the slider to select a year to display
      8            the total emissions for each country. 
      9            Scroll down to see an interactive 3D representation.""")
     11 col1, col2 = st.columns(2)
---> 13 df_total = pd.read_csv('data/co2_total.csv')
     14 col = 'Annual CO₂ emissions'
     15 max = df_total[col].max()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/pandas/namespace.py:355, in read_csv(pat

In [0]:
# add/subtract from the selected countries
    c = st.multiselect('Add a country:', countries, default=['United States', 'China', 'Russia', 'Germany'])
    tab1, tab2 = col2.tabs(["Graph", "Table"])

    with tab1:
        # plot a line graph of emissions for selected countries
        fig = px.line(df_total[df_total['Entity'].isin(c)], x='Year', y='Annual CO₂ emissions', color = 'Entity')
        st.plotly_chart(fig, use_container_width=True)
    with tab2:
        table = df_total[df_total['Year']==st.session_state['year']]
        st.dataframe(table[table['Entity'].isin(c)], use_container_width=True)

  File <command-4483247640063524>, line 2
    c = st.multiselect('Add a country:', countries, default=['United States', 'China', 'Russia', 'Germany'])
    ^
IndentationError: unexpected indent
